# First try 

In [4]:
import pandas as pd
from pandas.api.types import CategoricalDtype
from datetime import datetime

# read the CSV file from the specified URL
characteristics = pd.read_csv("/Users/jensknudsen/Desktop/Large_files/datashare.csv")

# convert the DATE column to datetime and floor to the nearest month
characteristics["DATE"] = pd.to_datetime(characteristics["DATE"])
#characteristics["DATE"] = characteristics["DATE"].dt.to_period("M")
#
## rename the DATE column to "month" and rename all other columns to start with "characteristic_"
#characteristics = characteristics.rename(columns={"DATE": "month"})
#characteristics = characteristics.rename(columns=lambda col: f"characteristic_{col}" if col not in ["permno", "month", "sic2"] else col)

In [ ]:
characteristics.iloc[6]

permno                                10065
DATE          1970-01-01 00:00:00.019570329
mvel1                              76945.25
beta                               1.218324
betasq                             1.484313
                          ...              
retvol                             0.009285
std_dolvol                         0.585667
std_turn                           0.294445
zerotrade                               0.0
sic2                                    NaN
Name: 5, Length: 97, dtype: object

# Second try

In [4]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np
import sqlite3
import datetime

In [53]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("/Users/jensknudsen/Desktop/Seminar/tidy_finance_ML.sqlite")

# Create a pandas DataFrame from the "crsp_monthly" table in the SQLite database
crsp_monthly = pd.read_sql_query("SELECT permno, month, ret_excess, mktcap_lag, sic2, macro_dp, macro_bm, macro_ntis, macro_tbl, characteristic_mom1m, characteristic_mvel1, characteristic_mom12m, characteristic_chmom, characteristic_maxret FROM stock_characteristics_monthly", conn)

# define a function to convert the number of days to a datetime object
def convert_days_to_datetime(num_days):
    start_date = datetime.datetime(1960, 1, 1)
    delta = datetime.timedelta(days=num_days)
    return start_date + delta

# apply the function to the 'date' column to convert it to datetime
crsp_monthly['month'] = crsp_monthly['month'].apply(convert_days_to_datetime)
crsp_monthly
# Select only the stocks that have 708 observations
df = crsp_monthly.query('month >= "1960-01-01"') \
                   .groupby('permno') \
                   .filter(lambda x: len(x) >= 611)

# Count the number of unique stocks in the DataFrame
num_stocks = len(df['permno'].unique())

In [1]:
# new columns

In [26]:
# Select only the stocks that have 708 observations
data = crsp_monthly.query('month >= "1960-01-01"') \
                   .groupby('permno') \
                   .filter(lambda x: len(x) >= 611)

# Count the number of unique stocks in the DataFrame
num_stocks = len(data['permno'].unique())

num_stocks

194

In [75]:
# Select only the stocks that have 708 observations
data = crsp_monthly.query('month >= "1980-01-01"') \
                   .groupby('permno') \
                   .filter(lambda x: len(x) >= 371)

# Count the number of unique stocks in the DataFrame
num_stocks = len(data['permno'].unique())

data

,permno,month,ret_excess,mktcap_lag,sic2,macro_dp,macro_bm,macro_ntis,macro_tbl,characteristic_mom1m,characteristic_mvel1,characteristic_mom12m,characteristic_chmom,characteristic_maxret
3386,10026.0,1980-01-01,-0.157215,103.479750,20.0,-3.385519,0.414971,-0.013897,0.0764,-0.613866,0.346686,0.255449,-0.661283,0.603799
3387,10026.0,1980-02-01,-0.172367,87.801000,20.0,-3.386185,0.409173,-0.011729,0.0774,-0.617551,0.314665,0.074068,-0.870420,0.472443
3388,10026.0,1980-02-29,-0.020686,73.167500,20.0,-3.402375,0.471334,-0.010291,0.0790,-0.861568,0.248340,-0.077844,-0.858809,0.397640
3389,10026.0,1980-03-31,0.051071,72.208500,20.0,-3.365010,0.480284,-0.010149,0.0777,-0.277522,0.233107,-0.439356,-0.692040,-0.092492
3390,10026.0,1980-04-30,-0.061595,76.394500,20.0,-3.443003,0.443570,-0.002262,0.0774,0.691600,0.261824,-0.591096,-0.460373,0.375810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196631,92807.0,2010-07-01,-0.060562,52.563897,60.0,-4.008492,0.253848,-0.011998,0.0013,0.244599,-0.689065,-0.293025,-0.232198,0.171878
3196632,92807.0,2010-08-01,-0.027975,48.959328,60.0,-4.080892,0.235975,-0.008504,0.0010,-0.564961,-0.718761,-0.263504,-0.460219,0.102736
3196633,92807.0,2010-09-01,0.007068,47.594609,60.0,-4.045576,0.241482,-0.005698,0.0011,-0.516324,-0.728535,-0.498907,-0.670554,-0.615081
3196634,92807.0,2010-10-01,-0.008997,47.935789,60.0,-4.020767,0.253146,-0.001895,0.0010,0.460520,-0.728571,-0.535662,-0.059680,-0.477980


In [21]:
# Connect to the database
import sqlite3
import pandas as pd

conn = sqlite3.connect("/Users/jensknudsen/Desktop/Seminar/tidy_finance_ML.sqlite")

#crsp_monthly = connection.execute("SELECT * FROM crsp_monthly").fetchall()

# Read in stock characteristics monthly data
data = pd.read_sql_query("SELECT permno, month, ret_excess, mktcap_lag, sic2, macro_dp, macro_bm, macro_ntis, macro_tbl, characteristic_mom1m, characteristic_mvel1, characteristic_mom12m, characteristic_chmom, characteristic_maxret FROM stock_characteristics_monthly", conn)
# define a function to convert the number of days to a datetime object
def convert_days_to_datetime(num_days):
    start_date = datetime.datetime(1960, 1, 1)
    delta = datetime.timedelta(days=num_days)
    return start_date + delta

# apply the function to the 'date' column to convert it to datetime
data['month'] = data['month'].apply(convert_days_to_datetime)
# print the resulting dataframe
len(data['permno'].unique())
data

,permno,month,ret_excess,mktcap_lag,sic2,macro_dp,macro_bm,macro_ntis,macro_tbl,characteristic_mom1m,characteristic_mvel1,characteristic_mom12m,characteristic_chmom,characteristic_maxret
0,10000.0,1976-02-01,-0.262443,16.100000,39.0,-3.347659,0.536377,-0.017914,0.0706,0.002090,-0.375760,0.001234,0.017457,0.938929
1,10000.0,1976-02-29,0.359385,11.960000,39.0,-3.394107,0.519628,-0.016420,0.0656,-0.948373,-0.497130,-0.004919,0.035137,-0.138898
2,10000.0,1976-03-31,-0.103792,16.330000,39.0,-3.376537,0.529714,-0.024585,0.0606,0.932873,-0.402101,0.018147,0.036117,0.813365
3,10000.0,1976-04-30,-0.227556,15.172000,39.0,-3.422238,0.503541,-0.021872,0.0615,-0.668110,-0.436052,0.025871,0.026223,-0.683852
4,10000.0,1976-05-31,-0.010225,11.793859,39.0,-3.432951,0.499281,-0.012293,0.0621,-0.942191,-0.534518,0.018967,0.025992,0.724257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216890,93436.0,2010-07-01,0.324911,200844.670898,37.0,-4.008492,0.253848,-0.011998,0.0013,0.854007,0.991708,0.990166,0.880896,0.289687
3216891,93436.0,2010-08-01,0.741352,266639.296940,37.0,-4.080892,0.235975,-0.008504,0.0010,0.902515,0.994492,0.990146,-0.771168,0.725004
3216892,93436.0,2010-09-01,-0.139187,464339.067705,37.0,-4.045576,0.241482,-0.005698,0.0011,0.990154,0.997943,0.990160,0.838192,0.774464
3216893,93436.0,2010-10-01,-0.095599,406701.489258,37.0,-4.020767,0.253146,-0.001895,0.0010,-0.726131,0.997959,0.990175,0.884279,0.797313


In [20]:
3216895/24851

129.4473059434228

In [6]:
# Connect to the database
conn = sqlite3.connect("/Users/jensknudsen/Desktop/Seminar/tidy_finance.sqlite")

#crsp_monthly = connection.execute("SELECT * FROM crsp_monthly").fetchall()

# Read crsp_monthly table into a dataframe
crsp_monthly = pd.read_sql_query("SELECT * FROM crsp_monthly", conn)
# Read factors_ff_monthly table into a dataframe
#factors_ff_monthly = pd.read_sql_query("SELECT * FROM factors_ff_monthly", conn)
# convert the date column with format 5902 to datetime and the month column with format 5875 to datetime:
#crsp_monthly['date'] = pd.to_datetime(crsp_monthly['date'], format='%Y%m')
#crsp_monthly['month'] = pd.to_datetime(crsp_monthly['month'], format='%Y%m')
#crsp_monthly['date'] = pd.to_datetime(crsp_monthly['date'])
#crsp_monthly['month'] = pd.to_datetime(crsp_monthly['month'])

# convert the column date into datetime: 


# define a function to convert the number of days to a datetime object
def convert_days_to_datetime(num_days):
    start_date = datetime.datetime(1960, 1, 1)
    delta = datetime.timedelta(days=num_days)
    return start_date + delta

# apply the function to the 'date' column to convert it to datetime
crsp_monthly['date'] = crsp_monthly['date'].apply(convert_days_to_datetime)
crsp_monthly['month'] = crsp_monthly['month'].apply(convert_days_to_datetime)
# print the resulting dataframe
crsp_monthly


# I wish to collect the smallest and largest date in the date column:

#crsp_monthly['date'] = pd.to_datetime(crsp_monthly['date'],  format='%y%m')
#crsp_monthly

#crsp_monthly['month'] = pd.to_datetime(crsp_monthly['month'])
#month_13 = crsp_monthly['month'].apply(lambda x: x + relativedelta(months=+13)).to_frame(name='month_13')
#crsp_monthly_lags = crsp_monthly.join(month_13)


# Calculate new column month_13 and add to crsp_monthly
#crsp_monthly_lags = crsp_monthly.assign(month_13 = crsp_monthly['month'] + pd.Timedelta(13, unit='M'))

# Join crsp_monthly and crsp_monthly_lags on permno and month
#crsp_monthly = pd.merge(crsp_monthly, crsp_monthly_lags, how='inner', on=['permno', 'month'], suffixes=('', '_13'))

# Create momentum_lag and size_lag columns
#data_portfolios = crsp_monthly.assign(momentum_lag = crsp_monthly['mktcap'] / crsp_monthly['mktcap_13'],
#                                      size_lag = np.log(crsp_monthly['mktcap_lag']))

# Drop rows containing NaN values in columns containing the word "lag"
#data_portfolios.dropna(subset=data_portfolios.filter(regex='lag').columns, inplace=True)


,permno,date,month,ret,shrout,altprc,exchcd,siccd,mktcap,mktcap_lag,exchange,industry,ret_excess,gvkey
0,10000.0,1976-02-28,1976-02-01,-0.257143,3680000.0,-3.250000,3.0,3990.0,11.960000,16.100000,NASDAQ,Manufacturing,-0.262443,013007
1,10000.0,1976-03-30,1976-02-29,0.365385,3680000.0,-4.437500,3.0,3990.0,16.330000,11.960000,NASDAQ,Manufacturing,0.359385,013007
2,10000.0,1976-04-29,1976-03-31,-0.098592,3793000.0,-4.000000,3.0,3990.0,15.172000,16.330000,NASDAQ,Manufacturing,-0.103792,013007
3,10000.0,1976-05-29,1976-04-30,-0.222656,3793000.0,-3.109375,3.0,3990.0,11.793859,15.172000,NASDAQ,Manufacturing,-0.227556,013007
4,10000.0,1976-06-29,1976-05-31,-0.005025,3793000.0,-3.093750,3.0,3990.0,11.734594,11.793859,NASDAQ,Manufacturing,-0.010225,013007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3225156,93436.0,2010-08-31,2010-08-01,0.741452,931809000.0,498.320007,3.0,9999.0,464339.067705,266639.296940,NASDAQ,Public,0.741352,184996
3225157,93436.0,2010-09-30,2010-09-01,-0.139087,948000000.0,429.010010,3.0,9999.0,406701.489258,464339.067705,NASDAQ,Public,-0.139187,184996
3225158,93436.0,2010-10-30,2010-10-01,-0.095499,947901000.0,388.040009,3.0,9999.0,367823.512140,406701.489258,NASDAQ,Public,-0.095599,184996
3225159,93436.0,2010-11-30,2010-11-01,0.462736,947901000.0,567.599976,3.0,9999.0,538028.584458,367823.512140,NASDAQ,Public,0.462636,184996


# Example of LSTM

In [ ]:
import tensorflow as tf
import os
import pandas as pd
import numpy as np